## Step 1. 웹사이트에서 뉴스데이터 가져오기

#### 웹크롤링을 위한 Python Package를 import 해 줍니다.

In [1]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import pandas as pd

#### 아래의 URL은 NAVER 뉴스 경제면의 2017년 8월 30일자 금융 뉴스 첫번째 페이지 입니다.
http://news.naver.com/main/list.nhn?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20170830&page=1

In [2]:
url = 'http://news.naver.com/main/list.nhn?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20170830&page=1'

#### requests.get()으로 url을 요청하고, BeaurifulSoup를 활용하여 해당 HTML페이지를 파싱합니다.

In [3]:
r = requests.get(url)
soup = BeautifulSoup(r.text,'lxml')

#### 뉴스의 제목, 출처, 작성 일시 및 원문 하이퍼링크를 가져오는 과정을 진행합니다.

In [4]:
news_list = soup.find('div',{'class':'list_body'})
news_list = news_list.find_all('li')

#### 페이지 구조를 확인하여 필요한 데이터들이 어떤 class에 속해있는지 알아내고, soup.find() 또는 soup.find_all()로 찾아서 받아옵니다.

In [5]:
news_list[0]

<li>
<dl>
<dt>
<a href="http://news.naver.com/main/read.nhn?mode=LS2D&amp;mid=shm&amp;sid1=101&amp;sid2=259&amp;oid=005&amp;aid=0001022104">
								 공시위반 4개 법인 등에 과징금 3억 9200만원
								</a>
</dt>
<dd>
								금융위원회 증권선물위원회는 KD건설 등 법인 4개와 KD건설 이사 A씨에게 공시위반에 따른 과징금 총 3억9200만원을 부과했다 …<span class="writing">국민일보</span><span class="date">2017-08-30 21:46</span>
</dd>
</dl>
</li>

In [6]:
titles = []
for news in news_list:
    title = news.find('a').text.strip()
    if len(title) == 0:
        title = news.find('img')['alt']
    titles.append(title)
    
writers = [news.find('span',{'class':'writing'}).text for news in news_list]
dates = [news.find('span',{'class':'date'}).text for news in news_list]
href = [news.find('a')['href'] for news in news_list]

#### 받아온 데이터를 pandas Dataframe으로 정리합니다.

In [7]:
df = pd.DataFrame(columns=['제목','출처','날짜','원문링크'])
df['제목'] = titles
df['출처'] = writers
df['날짜'] = dates
df['원문링크'] = href

In [8]:
df.head()

,제목,출처,날짜,원문링크
0,공시위반 4개 법인 등에 과징금 3억 9200만원,국민일보,2017-08-30 21:46,http://news.naver.com/main/read.nhn?mode=LS2D&...
1,"박삼구측, 금호타이어 상표권 계약서 초안에 수정의견 제시",뉴시스,2017-08-30 21:41,http://news.naver.com/main/read.nhn?mode=LS2D&...
2,금융소비자 정보포털 '파인' 개설 1주년 맞아 새 단장,전자신문,2017-08-30 20:31,http://news.naver.com/main/read.nhn?mode=LS2D&...
3,7월 기업 직접금융 조달액 14조 4168억원…전월 比 0.6% 증가,헤럴드경제,2017-08-30 19:52,http://news.naver.com/main/read.nhn?mode=LS2D&...
4,"KIC, 내년부터 주주 의결권 행사한다",파이낸셜뉴스,2017-08-30 19:32,http://news.naver.com/main/read.nhn?mode=LS2D&...


#### 이제, 모든 페이지에 대해 기사를 받아와서 엑셀에 저장할 수 있도록 함수를 짜봅시다.

In [9]:
def getFinancialNews(page):
    url = 'http://news.naver.com/main/list.nhn?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20170830&page='
    r = requests.get(url + str(page))
    soup = BeautifulSoup(r.text,'lxml')
    
    news_list = soup.find('div',{'class':'list_body'})
    news_list = news_list.find_all('li')
    
    titles = []
    for news in news_list:
        title = news.find('a').text.strip()
        if len(title) == 0:
            title = news.find('img')['alt']
        titles.append(title)

    writers = [news.find('span',{'class':'writing'}).text for news in news_list]
    dates = [news.find('span',{'class':'date'}).text for news in news_list]
    href = [news.find('a')['href'] for news in news_list]
    
    df = pd.DataFrame(columns=['제목','출처','날짜','원문링크'])
    df['제목'] = titles
    df['출처'] = writers
    df['날짜'] = dates
    df['원문링크'] = href
    
    return df

In [10]:
def main(page):
    df_list = []
    for pg in range(1, page+1):
        df = getFinancialNews(pg)
        df_list.append(df)
    data = pd.concat(df_list)
    
    writer = pd.ExcelWriter('./Data/' + 'news_info_with_link.xlsx')
    data.to_excel(writer,'Naver Finance',index=False)
    writer.save()
    print('News data have been successfully imported.')
    return

#### 12페이지까지만 받아와서 저장합니다.

In [11]:
main(12)

News data have been successfully imported.
